In [2]:
import pandas as pd

In [26]:
greenTripData = pd.read_csv('./homework/nyc_taxi_raw_files/green_tripdata_2019-09.csv', nrows = 100)

In [10]:
zoneLookup = pd.read_csv('./homework/nyc_taxi_raw_files/taxi+_zone_lookup.csv', nrows=100)

In [13]:
## How many taxi trips were totally made on September 18th 2019?

In [27]:
greenTripData['lpep_pickup_timestamp'] = pd.to_datetime(greenTripData['lpep_pickup_datetime'])
greenTripData['lpep_dropoff_lpep_pickup_timestamp'] = pd.to_datetime(greenTripData['lpep_dropoff_datetime'])

# Convert datetime to date
greenTripData['lpep_pickup_date'] = greenTripData['lpep_pickup_timestamp'].dt.date
greenTripData['lpep_dropoff_date'] = greenTripData['lpep_dropoff_lpep_pickup_timestamp'].dt.date


In [28]:
target_date = pd.to_datetime('2019-09-18')
chunksize = 10 ** 6
with pd.read_csv('./homework/nyc_taxi_raw_files/green_tripdata_2019-09.csv', chunksize=chunksize) as reader:
    for chunk in reader:
        chunk['lpep_pickup_timestamp'] = pd.to_datetime(greenTripData['lpep_pickup_datetime'])
        chunk['lpep_dropoff_lpep_pickup_timestamp'] = pd.to_datetime(greenTripData['lpep_dropoff_datetime'])
        chunk['lpep_pickup_date'] = greenTripData['lpep_pickup_timestamp'].dt.date
        chunk['lpep_dropoff_date'] = greenTripData['lpep_dropoff_lpep_pickup_timestamp'].dt.date
        filtered_df = chunk[chunk['Timestamp'].dt.date == target_date.date()]


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,lpep_pickup_timestamp,lpep_dropoff_lpep_pickup_timestamp,lpep_pickup_date,lpep_dropoff_date
0,2,2019-09-01 00:10:53,2019-09-01 00:23:46,N,1,65,189,5,2.00,10.5,...,NaN,0.3,14.16,1,1,0.0,2019-09-01 00:10:53,2019-09-01 00:23:46,2019-09-01,2019-09-01
1,2,2019-09-01 00:31:22,2019-09-01 00:44:37,N,1,97,225,5,3.20,12.0,...,NaN,0.3,13.30,2,1,0.0,2019-09-01 00:31:22,2019-09-01 00:44:37,2019-09-01,2019-09-01
2,2,2019-09-01 00:50:24,2019-09-01 01:03:20,N,1,37,61,5,2.99,12.0,...,NaN,0.3,13.30,2,1,0.0,2019-09-01 00:50:24,2019-09-01 01:03:20,2019-09-01,2019-09-01
3,2,2019-09-01 00:27:06,2019-09-01 00:33:22,N,1,145,112,1,1.73,7.5,...,NaN,0.3,10.30,1,1,0.0,2019-09-01 00:27:06,2019-09-01 00:33:22,2019-09-01,2019-09-01
4,2,2019-09-01 00:43:23,2019-09-01 00:59:54,N,1,112,198,1,3.42,14.0,...,NaN,0.3,18.36,1,1,0.0,2019-09-01 00:43:23,2019-09-01 00:59:54,2019-09-01,2019-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,2019-09-01 00:40:55,2019-09-01 00:48:35,N,1,95,160,1,2.24,9.0,...,NaN,0.3,10.30,2,1,0.0,2019-09-01 00:40:55,2019-09-01 00:48:35,2019-09-01,2019-09-01
96,2,2019-09-01 00:13:52,2019-09-01 00:21:47,N,1,75,151,1,2.04,8.5,...,NaN,0.3,11.76,1,1,0.0,2019-09-01 00:13:52,2019-09-01 00:21:47,2019-09-01,2019-09-01
97,2,2019-09-01 00:37:24,2019-09-01 01:02:31,N,1,41,182,1,7.77,26.0,...,NaN,0.3,32.76,1,1,0.0,2019-09-01 00:37:24,2019-09-01 01:02:31,2019-09-01,2019-09-01
98,2,2019-09-01 00:22:15,2019-09-01 00:29:52,N,1,74,260,1,4.99,15.0,...,NaN,0.3,25.00,1,1,0.0,2019-09-01 00:22:15,2019-09-01 00:29:52,2019-09-01,2019-09-01
